# Notebook 03: Trend Analysis with Moving Averages

Welcome to Notebook 03! Moving Averages are one of the most popular and useful technical indicators.

## What You'll Learn

1. What are Moving Averages (MA)
2. Simple Moving Average (SMA) vs Exponential Moving Average (EMA)
3. Common MA periods (20, 50, 200)
4. Identifying trends with MAs
5. Golden Cross and Death Cross patterns
6. MA crossover trading signals

## Prerequisites

- Completed Notebooks 01-02
- Understanding of price charts

## Time Required

60-75 minutes

---

# Setup
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

# Try importing pandas_ta
try:
    import pandas_ta as ta
    print("✅ pandas-ta imported successfully!")
except ImportError:
    print("⚠️ pandas-ta not found. Install it with:")
    print("   pip install pandas-ta")
    print("\nYou need pandas-ta for technical indicators in this notebook.")

%matplotlib inline

# Try to use seaborn style, fall back to default if not available
try:
    plt.style.use('seaborn-v0_8-darkgrid')
except:
    try:
        plt.style.use('seaborn-darkgrid')
    except:
        plt.style.use('default')

print("✅ Libraries imported successfully!")

In [ ]:
# Fetch UUE data - 1 year for better MA visualization
ticker = "UUE.KL"

try:
    stock = yf.Ticker(ticker)
    data = stock.history(period="1y")
    
    if len(data) == 0:
        print(f"⚠️ No data returned for {ticker}")
        print("\n💡 Try these reliable alternatives:")
        print("   ticker = '1155.KL'  # Maybank")
        print("   ticker = '1295.KL'  # Public Bank")
        print("\nThen re-run this cell.")
    elif len(data) < 200:
        print(f"⚠️ Warning: Only {len(data)} days of data available.")
        print("SMA 200 requires 200+ days. Some indicators may show NaN values.")
        print(f"\n✅ Fetched {len(data)} days of data for {ticker}")
        print(f"📅 Date range: {data.index[0].strftime('%Y-%m-%d')} to {data.index[-1].strftime('%Y-%m-%d')}")
    else:
        print(f"✅ Fetched {len(data)} days of data for {ticker}")
        print(f"📅 Date range: {data.index[0].strftime('%Y-%m-%d')} to {data.index[-1].strftime('%Y-%m-%d')}")
except Exception as e:
    print(f"⚠️ Error fetching data: {e}")
    print("Please check your internet connection.")

In [ ]:
# Fetch UUE data - 1 year for better MA visualization
ticker = "UUE.KL"
stock = yf.Ticker(ticker)
data = stock.history(period="1y")

print(f"✅ Fetched {len(data)} days of data for {ticker}")
print(f"📅 Date range: {data.index[0].strftime('%Y-%m-%d')} to {data.index[-1].strftime('%Y-%m-%d')}")

## Part 2: Simple Moving Average (SMA)

The **Simple Moving Average** gives equal weight to all prices in the period.

### Common SMA Periods

- **SMA 20**: Short-term trend (roughly 1 month of trading days)
- **SMA 50**: Medium-term trend (roughly 2.5 months)
- **SMA 200**: Long-term trend (roughly 10 months)

### Interpretation

- **Price > MA**: Bullish (uptrend)
- **Price < MA**: Bearish (downtrend)
- **MA Slope Up**: Uptrend strength
- **MA Slope Down**: Downtrend strength

In [ ]:
# Calculate Simple Moving Averages
if 'data' not in locals() or len(data) == 0:
    print("⚠️ Please run the previous cells first to fetch stock data.")
else:
    data['SMA_20'] = data['Close'].rolling(window=20).mean()
    data['SMA_50'] = data['Close'].rolling(window=50).mean()
    data['SMA_200'] = data['Close'].rolling(window=200).mean()

    print("✅ SMAs calculated!\n")

    # Show latest values
    latest = data.iloc[-1]
    print("="*60)
    print(f"📊 Latest Values for {ticker}")
    print("="*60)
    print(f"\nCurrent Price: RM {latest['Close']:.2f}")
    print(f"SMA 20: RM {latest['SMA_20']:.2f}")
    print(f"SMA 50: RM {latest['SMA_50']:.2f}")
    print(f"SMA 200: RM {latest['SMA_200']:.2f}" if not pd.isna(latest['SMA_200']) else "SMA 200: N/A (need 200+ days)")

    # Determine trend
    print("\n📈 Trend Analysis:")
    if latest['Close'] > latest['SMA_20']:
        print("   Short-term (20-day): BULLISH ✅")
    else:
        print("   Short-term (20-day): BEARISH ⚠️")
        
    if latest['Close'] > latest['SMA_50']:
        print("   Medium-term (50-day): BULLISH ✅")
    else:
        print("   Medium-term (50-day): BEARISH ⚠️")

    if not pd.isna(latest['SMA_200']):
        if latest['Close'] > latest['SMA_200']:
            print("   Long-term (200-day): BULLISH ✅")
        else:
            print("   Long-term (200-day): BEARISH ⚠️")

In [ ]:
# Visualize price with SMAs
if 'data' not in locals() or len(data) == 0:
    print("⚠️ Please run the previous cells first to fetch stock data.")
elif 'SMA_20' not in data.columns:
    print("⚠️ Please run the previous cell first to calculate moving averages.")
else:
    fig = go.Figure()

    # Add price
    fig.add_trace(go.Scatter(
        x=data.index,
        y=data['Close'],
        name='Price',
        line=dict(color='#2E86AB', width=2)
    ))

    # Add SMAs
    fig.add_trace(go.Scatter(
        x=data.index,
        y=data['SMA_20'],
        name='SMA 20',
        line=dict(color='green', width=1.5, dash='dot')
    ))

    fig.add_trace(go.Scatter(
        x=data.index,
        y=data['SMA_50'],
        name='SMA 50',
        line=dict(color='orange', width=1.5, dash='dash')
    ))

    fig.add_trace(go.Scatter(
        x=data.index,
        y=data['SMA_200'],
        name='SMA 200',
        line=dict(color='red', width=2)
    ))

    fig.update_layout(
        title=f'{ticker} with Simple Moving Averages',
        yaxis_title='Price (RM)',
        xaxis_title='Date',
        template='plotly_white',
        height=600,
        hovermode='x unified'
    )

    fig.show()

    print("\n✅ Chart with SMAs created!")
    print("\n💡 Notice how:")
    print("   - SMA 20 follows price more closely (reacts faster)")
    print("   - SMA 200 is smoother (shows long-term trend)")
    print("   - MAs can act as support in uptrends")

## Part 3: Exponential Moving Average (EMA)

The **Exponential Moving Average** gives more weight to recent prices.

### SMA vs EMA

| Aspect | SMA | EMA |
|--------|-----|-----|
| **Weight** | Equal to all prices | More weight to recent prices |
| **Responsiveness** | Slower to react | Faster to react |
| **Lag** | More lag | Less lag |
| **Best For** | Long-term trends | Short-term trading |
| **Noise** | Filters more noise | Filters less noise |

### Which to Use?

- **Long-term investors**: Use SMA (smoother, less noise)
- **Active traders**: Use EMA (faster signals)
- **Best practice**: Use both and see what works for your style!

In [ ]:
# Calculate Exponential Moving Averages
data['EMA_20'] = data['Close'].ewm(span=20, adjust=False).mean()
data['EMA_50'] = data['Close'].ewm(span=50, adjust=False).mean()

print("✅ EMAs calculated!\n")

# Compare SMA vs EMA
latest = data.iloc[-1]
print("="*60)
print("📊 SMA vs EMA Comparison (20-period)")
print("="*60)
print(f"\nCurrent Price: RM {latest['Close']:.2f}")
print(f"SMA 20: RM {latest['SMA_20']:.2f}")
print(f"EMA 20: RM {latest['EMA_20']:.2f}")
print(f"\nDifference: RM {abs(latest['SMA_20'] - latest['EMA_20']):.3f}")

if latest['EMA_20'] > latest['SMA_20']:
    print("\n💡 EMA is above SMA - Recent prices are higher (bullish momentum)")
else:
    print("\n💡 EMA is below SMA - Recent prices are lower (bearish momentum)")

In [ ]:
# Compare SMA vs EMA visually
fig = make_subplots(
    rows=2, cols=1,
    subplot_titles=('20-Period Comparison', '50-Period Comparison'),
    vertical_spacing=0.1,
    shared_xaxes=True
)

# Top: 20-period
fig.add_trace(go.Scatter(x=data.index, y=data['Close'], name='Price', line=dict(color='blue')), row=1, col=1)
fig.add_trace(go.Scatter(x=data.index, y=data['SMA_20'], name='SMA 20', line=dict(color='green', dash='dash')), row=1, col=1)
fig.add_trace(go.Scatter(x=data.index, y=data['EMA_20'], name='EMA 20', line=dict(color='red', dash='dot')), row=1, col=1)

# Bottom: 50-period
fig.add_trace(go.Scatter(x=data.index, y=data['Close'], name='Price', line=dict(color='blue'), showlegend=False), row=2, col=1)
fig.add_trace(go.Scatter(x=data.index, y=data['SMA_50'], name='SMA 50', line=dict(color='green', dash='dash'), showlegend=False), row=2, col=1)
fig.add_trace(go.Scatter(x=data.index, y=data['EMA_50'], name='EMA 50', line=dict(color='red', dash='dot'), showlegend=False), row=2, col=1)

fig.update_layout(
    title=f'{ticker} - SMA vs EMA Comparison',
    height=800,
    template='plotly_white',
    hovermode='x unified'
)

fig.update_yaxes(title_text="Price (RM)", row=1, col=1)
fig.update_yaxes(title_text="Price (RM)", row=2, col=1)
fig.update_xaxes(title_text="Date", row=2, col=1)

fig.show()

print("\n✅ SMA vs EMA comparison chart created!")
print("\n💡 Key Observations:")
print("   - EMA (red) reacts faster to price changes")
print("   - SMA (green) is smoother and more stable")
print("   - EMA crosses price more frequently (more signals)")

## Part 4: Golden Cross and Death Cross

These are powerful long-term signals based on MA crossovers.

### Golden Cross 🌟
**Definition**: SMA 50 crosses ABOVE SMA 200
- **Signal**: BULLISH (strong buy signal)
- **Meaning**: Short-term trend is overtaking long-term trend upward
- **Action**: Consider buying or holding

### Death Cross ☠️
**Definition**: SMA 50 crosses BELOW SMA 200
- **Signal**: BEARISH (strong sell signal)
- **Meaning**: Short-term trend is overtaking long-term trend downward
- **Action**: Consider selling or avoiding

### Important Notes
- These are **lagging indicators** (they confirm trends, don't predict them)
- Best used in trending markets, not sideways markets
- Always confirm with other indicators

In [ ]:
# Detect Golden Cross and Death Cross
# A cross occurs when SMA 50 crosses SMA 200

# Calculate crossovers
data['SMA_50_above_200'] = data['SMA_50'] > data['SMA_200']
data['Crossover'] = data['SMA_50_above_200'].diff()

# Find Golden Crosses (1.0) and Death Crosses (-1.0)
golden_crosses = data[data['Crossover'] == 1.0].index
death_crosses = data[data['Crossover'] == -1.0].index

print("🌟 Golden Cross & Death Cross Analysis\n")
print("="*60)
print(f"\nGolden Crosses found: {len(golden_crosses)}")
if len(golden_crosses) > 0:
    print("Dates:")
    for date in golden_crosses:
        price_at_cross = data.loc[date, 'Close']
        print(f"   🌟 {date.strftime('%Y-%m-%d')} (Price: RM {price_at_cross:.2f})")

print(f"\nDeath Crosses found: {len(death_crosses)}")
if len(death_crosses) > 0:
    print("Dates:")
    for date in death_crosses:
        price_at_cross = data.loc[date, 'Close']
        print(f"   ☠️ {date.strftime('%Y-%m-%d')} (Price: RM {price_at_cross:.2f})")

# Current status
latest = data.iloc[-1]
if not pd.isna(latest['SMA_200']):
    print("\n📊 Current Status:")
    if latest['SMA_50'] > latest['SMA_200']:
        print("   🌟 Currently in GOLDEN CROSS territory (Bullish)")
    else:
        print("   ☠️ Currently in DEATH CROSS territory (Bearish)")
else:
    print("\n⚠️ Not enough data for 200-day SMA yet")

In [ ]:
# Visualize Golden Cross and Death Cross
fig = go.Figure()

# Add price
fig.add_trace(go.Scatter(
    x=data.index,
    y=data['Close'],
    name='Price',
    line=dict(color='blue', width=1.5)
))

# Add SMAs
fig.add_trace(go.Scatter(
    x=data.index,
    y=data['SMA_50'],
    name='SMA 50',
    line=dict(color='orange', width=2)
))

fig.add_trace(go.Scatter(
    x=data.index,
    y=data['SMA_200'],
    name='SMA 200',
    line=dict(color='red', width=2)
))

# Mark Golden Crosses
for date in golden_crosses:
    fig.add_vline(
        x=date,
        line_dash="dash",
        line_color="green",
        annotation_text="Golden Cross",
        annotation_position="top"
    )

# Mark Death Crosses
for date in death_crosses:
    fig.add_vline(
        x=date,
        line_dash="dash",
        line_color="darkred",
        annotation_text="Death Cross",
        annotation_position="bottom"
    )

fig.update_layout(
    title=f'{ticker} - Golden Cross & Death Cross Detection',
    yaxis_title='Price (RM)',
    xaxis_title='Date',
    template='plotly_white',
    height=600,
    hovermode='x unified'
)

fig.show()

print("\n✅ Golden Cross / Death Cross chart created!")

## Part 5: MA Crossover Trading Strategy

A simple but effective strategy using MA crossovers.

### Strategy Rules

**Buy Signal**:
- Short-term MA (e.g., SMA 20) crosses ABOVE long-term MA (e.g., SMA 50)
- Indicates upward momentum

**Sell Signal**:
- Short-term MA crosses BELOW long-term MA
- Indicates downward momentum

### Advantages
- Simple to understand
- Works well in trending markets
- Clear entry/exit signals

### Disadvantages
- Lagging indicator (misses early moves)
- Many false signals in sideways markets
- Needs confirmation from other indicators

In [ ]:
# Detect SMA 20/50 crossovers
data['SMA_20_above_50'] = data['SMA_20'] > data['SMA_50']
data['Signal'] = data['SMA_20_above_50'].diff()

# Buy signals (1.0) and Sell signals (-1.0)
buy_signals = data[data['Signal'] == 1.0].copy()
sell_signals = data[data['Signal'] == -1.0].copy()

print("📈 MA Crossover Trading Signals (SMA 20/50)\n")
print("="*60)
print(f"\nBuy Signals found: {len(buy_signals)}")
if len(buy_signals) > 0:
    print("\nRecent Buy Signals:")
    for date in buy_signals.tail(5).index:
        price = data.loc[date, 'Close']
        print(f"   🟢 BUY on {date.strftime('%Y-%m-%d')} at RM {price:.2f}")

print(f"\nSell Signals found: {len(sell_signals)}")
if len(sell_signals) > 0:
    print("\nRecent Sell Signals:")
    for date in sell_signals.tail(5).index:
        price = data.loc[date, 'Close']
        print(f"   🔴 SELL on {date.strftime('%Y-%m-%d')} at RM {price:.2f}")

# Calculate simple strategy performance
if len(buy_signals) > 0 and len(sell_signals) > 0:
    # Pair up buy and sell signals
    trades = []
    position = None
    
    for date in data.index:
        if date in buy_signals.index and position is None:
            position = {'buy_date': date, 'buy_price': data.loc[date, 'Close']}
        elif date in sell_signals.index and position is not None:
            position['sell_date'] = date
            position['sell_price'] = data.loc[date, 'Close']
            position['return'] = ((position['sell_price'] - position['buy_price']) / position['buy_price']) * 100
            trades.append(position)
            position = None
    
    if len(trades) > 0:
        print(f"\n\n💰 Strategy Performance:")
        print("="*60)
        print(f"Total completed trades: {len(trades)}")
        avg_return = np.mean([t['return'] for t in trades])
        winning_trades = len([t for t in trades if t['return'] > 0])
        win_rate = (winning_trades / len(trades)) * 100
        
        print(f"Average return per trade: {avg_return:.2f}%")
        print(f"Winning trades: {winning_trades}/{len(trades)} ({win_rate:.1f}%)")
        print(f"\nBest trade: +{max([t['return'] for t in trades]):.2f}%")
        print(f"Worst trade: {min([t['return'] for t in trades]):.2f}%")

In [ ]:
# Visualize trading signals
fig = go.Figure()

# Price
fig.add_trace(go.Scatter(
    x=data.index,
    y=data['Close'],
    name='Price',
    line=dict(color='blue', width=1.5)
))

# SMAs
fig.add_trace(go.Scatter(
    x=data.index,
    y=data['SMA_20'],
    name='SMA 20',
    line=dict(color='green', width=1.5, dash='dot')
))

fig.add_trace(go.Scatter(
    x=data.index,
    y=data['SMA_50'],
    name='SMA 50',
    line=dict(color='orange', width=1.5, dash='dash')
))

# Buy signals
if len(buy_signals) > 0:
    fig.add_trace(go.Scatter(
        x=buy_signals.index,
        y=buy_signals['Close'],
        mode='markers',
        name='Buy Signal',
        marker=dict(color='green', size=12, symbol='triangle-up', line=dict(color='darkgreen', width=2))
    ))

# Sell signals
if len(sell_signals) > 0:
    fig.add_trace(go.Scatter(
        x=sell_signals.index,
        y=sell_signals['Close'],
        mode='markers',
        name='Sell Signal',
        marker=dict(color='red', size=12, symbol='triangle-down', line=dict(color='darkred', width=2))
    ))

fig.update_layout(
    title=f'{ticker} - MA Crossover Trading Signals (SMA 20/50)',
    yaxis_title='Price (RM)',
    xaxis_title='Date',
    template='plotly_white',
    height=600,
    hovermode='x unified'
)

fig.show()

print("\n✅ Trading signals chart created!")
print("\n💡 Green triangles = Buy signals")
print("💡 Red triangles = Sell signals")

## Part 6: Key Takeaways

### ✅ What You Learned

1. **Moving Averages Basics**
   - MAs smooth price data
   - Help identify trends
   - Act as dynamic support/resistance

2. **SMA vs EMA**
   - SMA: Equal weight, smoother
   - EMA: More weight to recent prices, faster
   - Choose based on trading style

3. **Common Periods**
   - SMA/EMA 20: Short-term
   - SMA/EMA 50: Medium-term
   - SMA 200: Long-term

4. **Golden Cross & Death Cross**
   - Strong long-term signals
   - Confirm major trend changes
   - Use with other indicators

5. **MA Crossover Strategy**
   - Simple and effective
   - Works best in trending markets
   - Needs confirmation

### 🎯 Key Concepts

- **Price above MA** = Bullish
- **Price below MA** = Bearish
- **Fast MA crosses above slow MA** = Buy signal
- **Fast MA crosses below slow MA** = Sell signal

### 📚 Next Steps

In Notebook 04, we'll learn about:
- RSI (Relative Strength Index)
- MACD (Moving Average Convergence Divergence)
- Momentum indicators
- Overbought/Oversold conditions

## 💡 Practice Exercises

### Exercise 1: Different MA Periods
Try calculating:
- SMA 10 and SMA 30
- Compare with SMA 20 and SMA 50
- Which gives more signals?

### Exercise 2: EMA Crossover Strategy
Implement the same crossover strategy but using:
- EMA 12 and EMA 26 (popular day trading combination)
- Compare performance with SMA 20/50

### Exercise 3: Multiple Stocks
Compare MA trends for:
- UUE.KL
- Maybank (1155.KL)
- Which is in a stronger trend?

### Exercise 4: MA as Support
Identify times when:
- Price bounced off SMA 50 (support)
- Price was rejected by SMA 50 (resistance)

In [ ]:
# Your practice code here!
# Try implementing the exercises above

---

## 🚀 Ready for More?

Excellent work! You now understand one of the most important technical indicators.

Proceed to **Notebook 04: Momentum Indicators (RSI & MACD)**

You'll learn to identify overbought and oversold conditions!

---

**Happy Learning! 📚📈**

*Remember: Moving averages are lagging indicators. They confirm trends but don't predict reversals.*